In [2]:
# imports
import nltk
import pandas as pd
import numpy as np
import artm

from nltk.corpus import brown
from nltk.stem import SnowballStemmer
from nltk.util import ngrams

In [ ]:
# work environment setup - one-time actions
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# MAKE STOP-WORDS DATASET
def make_stop_words_dataset():
    stemmer = SnowballStemmer('russian')
    stop_words= nltk.corpus.stopwords.words('russian')
    stop_words = list(map(lambda x: stemmer.stem(x), stop_words))
    pd.DataFrame(stop_words).to_csv('stop-words-russian-nltk.csv', index=False, header=['stop-word'])


make_stop_words_dataset()

# stop_word=" "
# for i in stop_words:
#         stop_word=  stop_word+" "+i
# print(stop_word)

In [ ]:
# DATA PREPARATION
# load raw data
def load_data():
    NROWS = None # 1000
    df = pd.read_csv(
        'vk.csv',
        nrows=NROWS, 
        low_memory=False, 
        dtype={'question': 'str',
               'answer':'str'})
    return df


# stemming
def stemming(df):
    df['question_stem'] = df['question'].apply(lambda x: ' '.join([stemmer.stem(w) for w in str(x).split()]))
    df['answer_stem'] = df['answer'].apply(lambda x: ' '.join([stemmer.stem(w) for w in str(x).split()]))
    # df[['question_stem','answer_stem']]
    # df.to_csv('vk-stemmed.csv', index=False, header=True)

    # remove punctuation & stop-words
    stop_words = list(pd.read_csv('stop-words-russian-nltk.csv')['stop-word'])


def clean_text(s):
    words = nltk.word_tokenize(s)
    words = [w for w in words if w not in stop_words]  # remove stop-words
    words = [w for w in words if w.isalpha() and len(w)>1]  # remove numbers and single-char
    return ' '.join(words)


def data_preparation():
    df = load_data()
    stemming(df)
    
    df['question_clean'] = df['question_stem'].apply(lambda x: clean_text(x))
    df['answer_clean'] = df['answer_stem'].apply(lambda x: clean_text(x))
    df.to_csv('vk-cleaned.csv', index=False, header=True, columns=['question_clean','answer_clean'])


data_preparation()

In [3]:
df = pd.read_csv('vk-cleaned.csv')
df.head()

,question_clean,answer_clean
0,кредитн карт моментум расплачива зарубежн инте...,можете использова кредит карт cred momentum оп...
1,дебетн карт пришл сообщен неразрешен задолженн...,дан вопрос обрат служб помощ банк телефон спец...
2,вопрос поч приход смс пополнен карты рад подкл...,дан вопрос обрат служб помощ банк телефон спец...
3,зарплатн карт сб польз сбол кром перевод частн...,дан вопрос обрат служб помощ банк телефон спец...
4,пришл смс одобр предворительн кредит документ ...,потребительск кред выда соблюден след условий ...


In [19]:
df['index'] = df.index.values
df['index'] = df['index'].apply(lambda x: str(x))
df['vw'] = 'd' + df['index'] + ' ' + df['question_clean'] + ' ' + df['answer_clean']
df[['vw']].to_csv('vk.vw', header=False, index=False)

In [21]:
# create ARTM batches & dictionary - one-time action
batch_vectorizer = artm.BatchVectorizer(
    data_path = 'vk.vw',
    data_format='vowpal_wabbit',
    target_folder='artm')

dictionary = artm.Dictionary()
dictionary.gather(data_path='artm')
dictionary.save(dictionary_path='artm/dictionary')
dictionary.save_text(dictionary_path='artm/dictionary.txt')

In [22]:
batch_vectorizer = artm.BatchVectorizer(data_path='artm',
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path='artm/dictionary.dict')
model = artm.ARTM(num_topics=200, dictionary=dictionary, cache_theta=True)
model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=dictionary))
model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
model.scores.add(artm.TopTokensScore(name='top_tokens_score'))
model.num_document_passes = 5

In [23]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=25)
#model.score_tracker['perplexity_score'].last_value


In [32]:
print(model.score_tracker['perplexity_score'].value)      # .last_value
print()
print(model.score_tracker['sparsity_phi_score'].value)    # .last_value
print()
print(model.score_tracker['sparsity_theta_score'].value)  # .last_value

[34506.7265625, 1237.1531982421875, 967.7527465820312, 659.1366577148438, 463.2226867675781, 369.8486328125, 317.751220703125, 283.50244140625, 259.8160095214844, 243.5580291748047, 232.4199981689453, 224.60557556152344, 218.74485778808594, 214.09605407714844, 210.3234405517578, 207.36085510253906, 205.0880584716797, 203.30642700195312, 201.82235717773438, 200.53749084472656, 199.4772491455078, 198.61947631835938, 197.9143524169922, 197.33203125, 196.8270263671875]

[0.0, 4.1600799249863485e-07, 0.0001307651837123558, 0.010894694365561008, 0.07629738748073578, 0.23459522426128387, 0.4166997969150543, 0.557007372379303, 0.6490408182144165, 0.7088770270347595, 0.7503583431243896, 0.7818784117698669, 0.8076567649841309, 0.8294609785079956, 0.8482967019081116, 0.8646963238716125, 0.8789665102958679, 0.8915693163871765, 0.9026430249214172, 0.912285566329956, 0.9208088517189026, 0.9281775951385498, 0.9347061514854431, 0.9404106140136719, 0.9454201459884644]

[0.0, 0.0, 0.0, 4.985417376701662

In [31]:
for topic_name in model.topic_names:
    print(
        topic_name + ':',
        model.score_tracker['top_tokens_score'].last_tokens[topic_name]
    )

topic_0: ['собствен', 'квартир', 'объект', 'ипотек', 'ипотеку', 'ипотечн', 'недвижимости', 'недвижим', 'залог', 'созаемщик']
topic_1: ['пожалуйста', 'заблокирова', 'сожалению', 'мож', 'сразу', 'заблокир', 'вопросов', 'лс', 'дожд', 'очереди']
topic_2: ['отказ', 'отказа', 'связа', 'повторн', 'выдач', 'банк', 'реша', 'различн', 'положительн', 'означает']
topic_3: ['смс', 'вчер', 'было', 'утр', 'мн', 'пришло', 'ноч', 'подруг', 'банк', 'поступит']
topic_4: ['наш', 'банк', 'сайте', 'http', 'завест', 'рекоменду', 'подойдет', 'сбере', 'подпиш', 'ознаком']
topic_5: ['http', 'сайт', 'ознаком', 'ссылке', 'тариф', 'банк', 'ряд', 'картами', 'тип', 'горазд']
topic_6: ['выпуск', 'регион', 'москв', 'региона', 'доставк', 'дизайн', 'планир', 'карт', 'стандартн', 'котор']
topic_7: ['онлайн', 'сбербанк', 'посмотрет', 'функц', 'распечата', 'системе', 'происходит', 'допуст', 'рекомендации', 'увидет']
topic_8: ['деньг', 'списа', 'полож', 'пополн', 'быть', 'перечисл', 'перевед', 'ушл', 'мир', 'чуж']
topic_9: 

topic_72: ['открыт', 'счета', 'сберкнижк', 'открыва', 'котор', 'вклады', 'пластиков', 'зелен', 'сберкнижки', 'улица']
topic_73: ['банкомат', 'оста', 'нем', 'вопросы', 'приостанов', 'правд', 'круглосуточн', 'банк', 'бланк', 'обстоятельств']
topic_74: ['действ', 'срок', 'автоматическ', 'действия', 'истечен', 'окончан', 'перевыпуска', 'нов', 'выпущен', 'перевыпущен']
topic_75: ['ситуац', 'приход', 'причин', 'сообщен', 'поэт', 'телефону', 'банк', 'несколько', 'исправлен', 'рекоменду']
topic_76: ['получател', 'наименован', 'банк', 'имя', 'представл', 'код', 'отчеств', 'реквизит', 'sberbank', 'swift']
topic_77: ['месяц', 'назад', 'втор', 'касс', 'внос', 'днях', 'паспорту', 'лож', 'неделю', 'половин']
topic_78: ['последн', 'полн', 'цифр', 'пакет', 'хххх', 'текстом', 'экономн', 'направ', 'тариф', 'четыр']
topic_79: ['рекоменду', 'обрат', 'консультац', 'урегулирован', 'возникл', 'трудност', 'обратиться', 'котор', 'проблемн', 'банк']
topic_80: ['сберегательн', 'отсутств', 'книжк', 'сертификат', 

topic_142: ['адрес', 'электрон', 'отправ', 'предложен', 'ул', 'адресу', 'москва', 'способами', 'постав', 'отзыв']
topic_143: ['перв', 'прошл', 'выполн', 'очеред', 'отправля', 'поступили', 'прошел', 'пар', 'недели', 'обращаться']
topic_144: ['возврат', 'взял', 'полност', 'возвраща', 'вернут', 'погаш', 'страховк', 'согласия', 'возврата', 'навяза']
topic_145: ['мог', 'хотел', 'узнать', 'здраствуйте', 'карточку', 'возможным', 'мастер', 'небольш', 'кард', 'примеру']
topic_146: ['производ', 'сраз', 'предъявлен', 'количеств', 'котор', 'фактическ', 'ранее', 'предусмотрен', 'транзакц', 'разниц']
topic_147: ['интернет', 'покупк', 'оплачива', 'магазин', 'товар', 'покупок', 'интернете', 'сет', 'классическ', 'технолог']
topic_148: ['заявлен', 'оформ', 'спорн', 'придт', 'обрат', 'рекоменду', 'банк', 'необходим', 'обычную', 'котор']
topic_149: ['решен', 'принима', 'банк', 'сбербанке', 'принят', 'продукт', 'порядке', 'одобр', 'учитыва', 'расчет']
topic_150: ['денег', 'времен', 'делать', 'карточк', 'су

In [28]:
model.score_tracker

{'perplexity_score': <artm.score_tracker.PerplexityScoreTracker at 0x26542e10>,
 'sparsity_phi_score': <artm.score_tracker.SparsityPhiScoreTracker at 0x26542e48>,
 'sparsity_theta_score': <artm.score_tracker.SparsityThetaScoreTracker at 0x26542e80>,
 'top_tokens_score': <artm.score_tracker.TopTokensScoreTracker at 0x26542ef0>}

In [ ]:
# def process_ngrams(s):
#     tokens = nltk.word_tokenize(s)
    

# df['question_ngram'] = dfp['question_stem'].apply(
#     lambda x: process_ngrams(x)
# )